In [1]:
import pandas as pd
import json
from pathlib import Path

In [2]:
df = pd.read_csv("netflix_titles.csv")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
output_dir = Path("../data")
output_dir.mkdir(exist_ok=True)
df.to_json(output_dir / "netflix_raw.json", orient="records", indent=2)

In [4]:
# TOP CONTENT
import numpy as np

enhanced_df = df.copy()

np.random.seed(42)

enhanced_df["views"] = np.random.randint(10_000_000, 200_000_000, size=len(df))
enhanced_df["completionRate"] = np.random.randint(60, 100, size=len(df))
enhanced_df["avgWatchTime"] = np.random.randint(30, 150, size=len(df))
enhanced_df["ratingSynthetic"] = np.round(np.random.uniform(3.0, 5.0, size=len(df)), 1)

threshold = enhanced_df["views"].quantile(0.9)
enhanced_df["trending"] = enhanced_df["views"] >= threshold

enhanced_df["genre"] = enhanced_df["listed_in"].fillna("").str.split(", ").str[0]
enhanced_df["releaseDate"] = enhanced_df["date_added"].fillna("")

top_content_full = enhanced_df.sort_values("views", ascending=False).head(100)[[
    "show_id",
    "title",
    "type",
    "genre",
    "releaseDate",
    "views",
    "completionRate",
    "avgWatchTime",
    "ratingSynthetic",
    "trending",
]]

top_content_full = top_content_full.rename(columns={
    "show_id": "id",
    "ratingSynthetic": "rating"
})

top_content_full.to_json("../data/topContent.json", orient="records", indent=2)

len(top_content_full)

100

In [7]:
# VIEWERSHIP TREND
import pandas as pd
import json
import re

months = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]

view_df = pd.read_csv("netflix_titles.csv")
view_df['date_added'] = pd.to_datetime(view_df['date_added'], errors='coerce')
view_df = view_df.dropna(subset=['date_added'])
view_df['month'] = view_df['date_added'].dt.strftime('%b')

# Extract numeric duration
def parse_duration(x):
    if pd.isna(x):
        return None
    if "Season" in x:
        return None  # ignore series for avgDuration
    match = re.search(r'(\d+)', str(x))
    if match:
        return int(match.group(1))
    return None

view_df['duration_min'] = view_df['duration'].apply(parse_duration)

# Placeholder views
view_df['views'] = 1_000_000

monthly = view_df.groupby('month').agg(
    views=('views', 'sum'),
    uniqueViewers=('views', lambda x: int(x.sum() * 0.72)),
    avgDuration=('duration_min', lambda x: int(x.dropna().mean() if len(x.dropna()) > 0 else 45))
)

monthly = monthly.reindex(months, fill_value=0)

viewershipTrend = monthly.reset_index().to_dict(orient='records')

with open("../data/viewershipTrend.json", "w") as f:
    json.dump(viewershipTrend, f, indent=2)

viewershipTrend


[{'month': 'Jan',
  'views': 727000000,
  'uniqueViewers': 523440000,
  'avgDuration': 99},
 {'month': 'Feb',
  'views': 557000000,
  'uniqueViewers': 401040000,
  'avgDuration': 96},
 {'month': 'Mar',
  'views': 734000000,
  'uniqueViewers': 528480000,
  'avgDuration': 99},
 {'month': 'Apr',
  'views': 759000000,
  'uniqueViewers': 546480000,
  'avgDuration': 102},
 {'month': 'May',
  'views': 626000000,
  'uniqueViewers': 450720000,
  'avgDuration': 98},
 {'month': 'Jun',
  'views': 724000000,
  'uniqueViewers': 521280000,
  'avgDuration': 98},
 {'month': 'Jul',
  'views': 819000000,
  'uniqueViewers': 589680000,
  'avgDuration': 98},
 {'month': 'Aug',
  'views': 749000000,
  'uniqueViewers': 539280000,
  'avgDuration': 100},
 {'month': 'Sep',
  'views': 765000000,
  'uniqueViewers': 550800000,
  'avgDuration': 100},
 {'month': 'Oct',
  'views': 755000000,
  'uniqueViewers': 543600000,
  'avgDuration': 98},
 {'month': 'Nov',
  'views': 697000000,
  'uniqueViewers': 501840000,
  'avgD

In [ ]:
# GENRE DISTRIBUTION
from collections import Counter

genres = df['listed_in'].dropna().str.split(', ')
all_genres = [g for sub in genres for g in sub]
genre_counts = Counter(all_genres)

genre_distribution = [
    {"name": genre, "value": count}
    for genre, count in genre_counts.most_common()
]

with open("../data/genreDistribution.json", "w") as f:
    json.dump(genre_distribution, f, indent=2)

genre_distribution[:10]

[{'name': 'International Movies', 'value': 2752},
 {'name': 'Dramas', 'value': 2427},
 {'name': 'Comedies', 'value': 1674},
 {'name': 'International TV Shows', 'value': 1351},
 {'name': 'Documentaries', 'value': 869},
 {'name': 'Action & Adventure', 'value': 859},
 {'name': 'TV Dramas', 'value': 763},
 {'name': 'Independent Movies', 'value': 756},
 {'name': 'Children & Family Movies', 'value': 641},
 {'name': 'Romantic Movies', 'value': 616}]

In [8]:
# KEY INSIGHTS
keyInsights = [
    {
        "title": "Record-Breaking Month",
        "description": f"Total views hit {viewershipTrend[-1]['views']:,} in December.",
        "change": 15,
        "metric": "vs. last month"
    },
    {
        "title": "Engagement Up",
        "description": f"Average completion rate across titles reached {int(enhanced_df['completionRate'].mean())}%.",
        "change": 7,
        "metric": "vs. quarterly avg"
    },
    {
        "title": "Drama Leads",
        "description": "Drama remains the most consumed category across all regions.",
        "change": 5,
        "metric": "share growth"
    }
]

with open("../data/keyInsights.json", "w") as f:
    json.dump(keyInsights, f, indent=2)

keyInsights

[{'title': 'Record-Breaking Month',
  'description': 'Total views hit 75,200,000 in December.',
  'change': 15,
  'metric': 'vs. last month'},
 {'title': 'Engagement Up',
  'description': 'Average completion rate across titles reached 79%.',
  'change': 7,
  'metric': 'vs. quarterly avg'},
 {'title': 'Drama Leads',
  'description': 'Drama remains the most consumed category across all regions.',
  'change': 5,
  'metric': 'share growth'}]

In [12]:
# KPI METRICS
kpiMetrics = {
    "totalViews": int(enhanced_df["views"].sum()),
    "totalViewsChange": 10.5,
    "avgCompletionRate": float(round(enhanced_df["completionRate"].mean(), 1)),
    "completionChange": 3.2, 
    "avgWatchTime": int(enhanced_df["avgWatchTime"].mean()),
    "watchTimeChange": 9.8,
    "activeSubscribers": int(enhanced_df["views"].sum() / 2),
    "subscriberChange": 6.1
}

with open("../data/kpiMetrics.json", "w") as f:
    json.dump(kpiMetrics, f, indent=2)

kpiMetrics

{'totalViews': 921895236828,
 'totalViewsChange': 10.5,
 'avgCompletionRate': 79.3,
 'completionChange': 3.2,
 'avgWatchTime': 89,
 'watchTimeChange': 9.8,
 'activeSubscribers': 460947618414,
 'subscriberChange': 6.1}